In [1]:
import os
import pandas as pd

## Reading dataset

In [27]:
path = "archive/20news-19997/20_newsgroups"
df = []

for folder in os.listdir(path):
    for file in os.listdir(f'{path}/{folder}'):
        f = open(f'{path}/{folder}/{file}', 'rb')
        df.append(f.read())

In [28]:
df = pd.DataFrame(df)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19997 entries, 0 to 19996
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       19997 non-null  object
dtypes: object(1)
memory usage: 156.4+ KB


In [29]:
df.head()

,0
0,"b""Newsgroups: sci.space\nPath: cantaloupe.srv...."
1,b'Newsgroups: sci.space\nPath: cantaloupe.srv....
2,"b""Newsgroups: sci.space\nPath: cantaloupe.srv...."
3,b'Newsgroups: sci.space\nPath: cantaloupe.srv....
4,b'Newsgroups: sci.space\nPath: cantaloupe.srv....


In [30]:
df[0][0]

b"Newsgroups: sci.space\nPath: cantaloupe.srv.cs.cmu.edu!rochester!udel!bogus.sura.net!news-feed-1.peachnet.edu!gatech!swrinde!sdd.hp.com!ux1.cso.uiuc.edu!news.cso.uiuc.edu!uxa.cso.uiuc.edu!gfk39017\nFrom: gfk39017@uxa.cso.uiuc.edu (George F. Krumins)\nSubject: Re: space news from Feb 15 AW&ST\nDate: Fri, 23 Apr 1993 20:16:24 GMT\nMessage-ID: <C5yDnC.GwB@news.cso.uiuc.edu>\nReferences: <C5ros0.uy@zoo.toronto.edu> <1993Apr23.155313.4220@dazixco.ingr.com>\nSender: usenet@news.cso.uiuc.edu (Net Noise owner)\nOrganization: University of Illinois at Urbana\nLines: 23\n\njbreed@doink.b23b.ingr.com (James B. Reed) writes:\n\n>In article <C5ros0.uy@zoo.toronto.edu>, henry@zoo.toronto.edu (Henry Spencer) writes:\n>|> [Pluto's] atmosphere will start to freeze out around 2010, and after about\n>|> 2005 increasing areas of both Pluto and Charon will be in permanent\n>|> shadow that will make imaging and geochemical mapping impossible.\n\nIt's my understanding that the freezing will start to occur 

## Converting bytes to string

In [ ]:
def bytes_to_str(text):
    return text.decode('utf-8', errors = 'ignore')
df[0]=df[0].apply(bytes_to_str)

## Removing headers

In [ ]:
def remove_header(text):
    split_text = text.split('\n\n', 1)
    if len(split_text) > 1:
        return split_text[1]
    return text
df[0] = df[0].apply(remove_header)

In [36]:
df[0][127]

'>I was reading Popular Science this morning and was surprised by an ad in\n>the back.  I know that a lot of the ads in the back of PS are fringe\n>science or questionablely legal, but this one really grabbed my attention.\n>It was from a company name "Personal Missle, Inc." or something like that.\n\nThe company was probably "Public Missiles, Inc" of Michigan.\n\n>Anyhow, the ad stated that they\'d sell rockets that were up to 20\' in length\n>and engines of sizes "F" to "M".  They also said that some rockets will\n>reach 50,000 feet.\n\nYup.\n\n>Now, aside from the obvious dangers to any amateur rocketeer using one\n>of these beasts, isn\'t this illegal?  I can\'t imagine the FAA allowing\n>people to shoot rockets up through the flight levels of passenger planes.\n>Not to even mention the problem of locating a rocket when it comes down.\n\nNope, it\'s not illegal. It is, however, closely regulated. In order to \npurchase and use the big rocket motors required, it is necessary to be\n

## Dividing into train, test and valid datasets

In [38]:
from sklearn.model_selection import train_test_split

print(df.shape)

X_build, X_val = train_test_split(
    df,
    test_size=0.3, random_state=213)

X_train, X_test = train_test_split(
    X_build,
    test_size=0.3, random_state=7)

print("X_train shape: {}".format(X_train.shape))
print("X_test shape: {}".format(X_test.shape))
print("X_val shape: {}".format(X_val.shape))

(19997, 1)
X_train shape: (9797, 1)
X_test shape: (4200, 1)
X_val shape: (6000, 1)
